In [20]:
# base imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Важная настройка для корректной настройки pipeline!
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# for model learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import category_encoders as ce

# Models
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as lgb # устоявшееся соглашение импорта
from lightgbm import LGBMRegressor

# Metrics
from sklearn.metrics import mean_squared_log_error, mean_squared_error, mean_absolute_error, r2_score

# Tuning parametrs
import optuna
from sklearn.model_selection import KFold

# notebook settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Импорт пайплайна препроцессинга

In [3]:
from custom_process import MyTransformer, MissingValueReplacer

In [4]:
from joblib import load

preprocessing = load('preprocessing_pipeline.pkl')

### 0. Загрузка датасета

In [17]:
data = pd.read_csv('/Users/alex_eyfler/ds_bootcamp/first-phase-projects/house_prices_kaggle_regression/original data/train.csv')
X_df = preprocessing.transform(data)
df = pd.concat([X_df, data['SalePrice']], axis=1)

X, y_log = df.drop('SalePrice', axis=1), np.log(df['SalePrice'])

X_train, X_valid, y_log_train, y_log_valid = train_test_split(
    X,
    y_log,
    test_size=0.2,
    random_state=52
)

/Users/alex_eyfler/ds_bootcamp/first-phase-projects/house_prices_kaggle_regression/Lambert/custom_process.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[columns_with_values] = X[columns_with_values].replace(replace_map)
/Users/alex_eyfler/ds_bootcamp/first-phase-projects/house_prices_kaggle_regression/env/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/Users/alex_eyfler/ds_bootcamp/first-phase-projects/h

### 1. Пробуем CatBoost с базовыми настройками

In [18]:
cat_boost_1 = CatBoostRegressor()

cat_boost_1.fit(
    X_train,
    y_log_train
)

Learning rate set to 0.04196
0:	learn: 0.3795880	total: 61.6ms	remaining: 1m 1s
1:	learn: 0.3701026	total: 63ms	remaining: 31.4s
2:	learn: 0.3606640	total: 64.2ms	remaining: 21.3s
3:	learn: 0.3513891	total: 65.6ms	remaining: 16.3s
4:	learn: 0.3422470	total: 66.7ms	remaining: 13.3s
5:	learn: 0.3339571	total: 68.4ms	remaining: 11.3s
6:	learn: 0.3253107	total: 69.6ms	remaining: 9.88s
7:	learn: 0.3166813	total: 70.8ms	remaining: 8.77s
8:	learn: 0.3092280	total: 71.8ms	remaining: 7.91s
9:	learn: 0.3017018	total: 73ms	remaining: 7.22s
10:	learn: 0.2942890	total: 74.4ms	remaining: 6.69s
11:	learn: 0.2877308	total: 75.8ms	remaining: 6.24s
12:	learn: 0.2806799	total: 77.7ms	remaining: 5.9s
13:	learn: 0.2748413	total: 79.1ms	remaining: 5.57s
14:	learn: 0.2688649	total: 80.3ms	remaining: 5.28s
15:	learn: 0.2625957	total: 81.5ms	remaining: 5.01s
16:	learn: 0.2563005	total: 82.8ms	remaining: 4.79s
17:	learn: 0.2509871	total: 83.9ms	remaining: 4.58s
18:	learn: 0.2458817	total: 85.7ms	remaining: 4.42

#### 1.1 Получение предсказаний и обработка метрик

In [19]:
# Предсказания на логарифмических данных
y_log_train_pred = cat_boost_1.predict(X_train)
y_log_valid_pred = cat_boost_1.predict(X_valid)

# Перевод предсказанных значений в обычную шкалу для дальнейшего анализа
y_train_pred = np.exp(y_log_train_pred)
y_valid_pred = np.exp(y_log_valid_pred)

# И для оригинальных значений
y_train = np.exp(y_log_train)
y_valid = np.exp(y_log_valid)


# Проверяем, что в целевых значениях и предсказаниях нет отрицательных значений
if (y_train < 0).any() or (y_valid < 0).any() or (y_train_pred < 0).any() or (y_valid_pred < 0).any():
    print("RMSLE не может быть применена, так как имеются отрицательные значения в y или в предсказаниях.")
else:
    # Вычисляем MSLE и RMSLE
    train_msle = mean_squared_log_error(y_train, y_train_pred)
    valid_msle = mean_squared_log_error(y_valid, y_valid_pred)

    print('Train MSLE:', train_msle)
    print('Validation MSLE:', valid_msle)
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

    print('Train RMSLE:', np.sqrt(train_msle))
    print('Validation RMSLE:', np.sqrt(valid_msle))
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Вычисляем другие метрики для более полной оценки модели

# Среднеквадратичная ошибка (MSE) и корень из MSE (RMSE)
train_mse = mean_squared_error(y_train, y_train_pred)
valid_mse = mean_squared_error(y_valid, y_valid_pred)

print('Train MSE:', train_mse)
print('Validation MSE:', valid_mse)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

print('Train RMSE:', np.sqrt(train_mse))
print('Validation RMSE:', np.sqrt(valid_mse))
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Средняя абсолютная ошибка (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_valid, y_valid_pred)

print('Train MAE:', train_mae)
print('Validation MAE:', valid_mae)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Коэффициент детерминации (R²)
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_valid, y_valid_pred)

print('Train R^2:', train_r2)
print('Validation R^2:', valid_r2)

Train MSLE: 0.0011406710710768903
Validation MSLE: 0.016886190005431623


Train RMSLE: 0.0337738222752014
Validation RMSLE: 0.1299468737809095


Train MSE: 40223732.22554607
Validation MSE: 716859264.9293038


Train RMSE: 6342.218241715281
Validation RMSE: 26774.227625261272


Train MAE: 4597.290519498844
Validation MAE: 15062.138643641978


Train R^2: 0.9931753197459061
Validation R^2: 0.9095432936357463


### 4. Тюнинг параметров для моделей

### 4.1 CatBoost тюнинг

In [ ]:
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
    }
    kf = KFold(n_splits=5, shuffle=True, random_state=52)
    rmsle_scores = []
    
    for train_index, valid_index in kf.split(X):
        X_train_cv, X_valid_cv = X.iloc[train_index], X.iloc[valid_index]
        y_train_cv, y_valid_cv = y.iloc[train_index], y.iloc[valid_index]
        
        model = CatBoostRegressor(**param)
        model.fit(X_train_cv, y_train_cv, eval_set=(X_valid_cv, y_valid_cv), verbose=False)
        
        y_pred = model.predict(X_valid_cv)
        y_pred = np.maximum(y_pred, 0)  # Убеждаемся, что прогнозы неотрицательны
        
        rmsle = np.sqrt(mean_squared_log_error(y_valid_cv, y_pred))
        rmsle_scores.append(rmsle)
    
    return np.mean(rmsle_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Наилучшие параметры:')
for key, value in study.best_params.items():
    print(f'{key}: {value}')

print(f'Наилучшее значение RMSLE: {study.best_value}')

### 5. Проверка моделей с оптимизированными параметрами

### 5.1 CatBoost оптимизированная

In [ ]:

# Наилучшие параметры: МОИ ПЕРВЫЕ
# iterations: 1842
# learning_rate: 0.148683039214686
# depth: 4
# l2_leaf_reg: 8.976684403702432
# bagging_temperature: 0.6928364121029715
# random_strength: 3.0234103345423176
# grow_policy: SymmetricTree
# Наилучшее значение RMSLE: 0.138473644119397

# Наилучшие параметры: МАРИНА ПРЕПРОЦЕССИНГ
# iterations: 900
# learning_rate: 0.03811379337443099
# depth: 4
# l2_leaf_reg: 5.401132284019045
# bagging_temperature: 0.7921960733675621
# random_strength: 7.0013473823336065
# grow_policy: Lossguide
# Наилучшее значение RMSLE: 0.1407241390521044

# Наилучшие параметры: ЛАМБЕРТ ПРЕПРОЦЕССИНГ
# iterations: 1241
# learning_rate: 0.06071716565899703
# depth: 4
# l2_leaf_reg: 7.76892727416148
# bagging_temperature: 0.961745094190735
# random_strength: 4.9804828019633165
# grow_policy: Lossguide
# Наилучшее значение RMSLE: 0.12333375686301161

In [ ]:
cat_boost_2 = CatBoostRegressor(
    iterations=1241,
    learning_rate=0.06071716565899703,
    depth=4,
    l2_leaf_reg=7.76892727416148,
    bagging_temperature=0.961745094190735,
    random_strength=4.9804828019633165,
    grow_policy='Lossguide'
)

cat_boost_2.fit(
    X_train,
    y_log_train
)

In [ ]:
# Предсказания на логарифмических данных
y_log_train_pred = cat_boost_2.predict(X_train)
y_log_valid_pred = cat_boost_2.predict(X_valid)

# Перевод предсказанных значений в обычную шкалу для дальнейшего анализа
y_train_pred = np.exp(y_log_train_pred)
y_valid_pred = np.exp(y_log_valid_pred)

# И для оригинальных значений
y_train = np.exp(y_log_train)
y_valid = np.exp(y_log_valid)


# Проверяем, что в целевых значениях и предсказаниях нет отрицательных значений
if (y_train < 0).any() or (y_valid < 0).any() or (y_train_pred < 0).any() or (y_valid_pred < 0).any():
    print("RMSLE не может быть применена, так как имеются отрицательные значения в y или в предсказаниях.")
else:
    # Вычисляем MSLE и RMSLE
    train_msle = mean_squared_log_error(y_train, y_train_pred)
    valid_msle = mean_squared_log_error(y_valid, y_valid_pred)

    print('Train MSLE:', train_msle)
    print('Validation MSLE:', valid_msle)
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

    print('Train RMSLE:', np.sqrt(train_msle))
    print('Validation RMSLE:', np.sqrt(valid_msle))
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Вычисляем другие метрики для более полной оценки модели

# Среднеквадратичная ошибка (MSE) и корень из MSE (RMSE)
train_mse = mean_squared_error(y_train, y_train_pred)
valid_mse = mean_squared_error(y_valid, y_valid_pred)

print('Train MSE:', train_mse)
print('Validation MSE:', valid_mse)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

print('Train RMSE:', np.sqrt(train_mse))
print('Validation RMSE:', np.sqrt(valid_mse))
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Средняя абсолютная ошибка (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_valid, y_valid_pred)

print('Train MAE:', train_mae)
print('Validation MAE:', valid_mae)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Коэффициент детерминации (R²)
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_valid, y_valid_pred)

print('Train R^2:', train_r2)
print('Validation R^2:', valid_r2)

# Добавляем графическое представление метрик

# 1. Фактические значения vs Предсказанные значения (Обучающая выборка)
plt.figure(figsize=(10, 6))
plt.scatter(y_train, y_train_pred, alpha=0.3)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title('Фактические vs Предсказанные значения (Обучающая выборка)')
plt.show()

# 2. Фактические значения vs Предсказанные значения (Валидационная выборка)
plt.figure(figsize=(10, 6))
plt.scatter(y_valid, y_valid_pred, alpha=0.3, color='orange')
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title('Фактические vs Предсказанные значения (Валидационная выборка)')
plt.show()

# 3. Ошибка предсказаний vs Предсказанные значения (Обучающая выборка)
train_residuals = y_train - y_train_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_train_pred, train_residuals, alpha=0.3)
plt.hlines(y=0, xmin=y_train_pred.min(), xmax=y_train_pred.max(), colors='r')
plt.xlabel('Предсказанные значения')
plt.ylabel('Ошибка предсказаний')
plt.title('Ошибка предсказаний vs Предсказанные значения (Обучающая выборка)')
plt.show()

# 4. Ошибка предсказаний vs Предсказанные значения (Валидационная выборка)
valid_residuals = y_valid - y_valid_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_valid_pred, valid_residuals, alpha=0.3, color='green')
plt.hlines(y=0, xmin=y_valid_pred.min(), xmax=y_valid_pred.max(), colors='r')
plt.xlabel('Предсказанные значения')
plt.ylabel('Ошибка предсказаний')
plt.title('Ошибка предсказаний vs Предсказанные значения (Валидационная выборка)')
plt.show()

# 5. Гистограмма распределения ошибки предсказаний (Обучающая выборка)
plt.figure(figsize=(10, 6))
sns.histplot(train_residuals, kde=True)
plt.xlabel('Ошибка предсказаний')
plt.title('Распределение ошибок предсказаний (Обучающая выборка)')
plt.show()

# 6. Гистограмма распределения остатков (Валидационная выборка)
plt.figure(figsize=(10, 6))
sns.histplot(valid_residuals, kde=True, color='purple')
plt.xlabel('Ошибки предсказаний')
plt.title('Распределение ошибок предсказаний (Валидационная выборка)')
plt.show()

### 6. Получение результатов и загрузка

In [16]:
# загрузка датасета для заполнения ответами
df_answer = pd.read_csv('../sample_submission.csv')

# загрузка датасета для проведения предсказания
df_test = pd.read_csv('/Users/alex_eyfler/ds_bootcamp/first-phase-projects/house_prices_kaggle_regression/Lambert/Lam_test.csv')

In [17]:
# Предсказания на логарифмических данных
y_log_test_pred = cat_boost_2.predict(df_test)

# Возвращаемся к исходной шкале
y_test_pred = np.exp(y_log_test_pred)

# Записываем ответы
df_answer['SalePrice'] = y_test_pred

# Сохраняем ответы для отправки на Kaggle
df_answer.to_csv('../submission_lambert.csv', index=False)

### 7. Сохранение модели

In [34]:
cat_boost_2.save_model('catboost_model_lam_data.cbm')